In [12]:
import numpy as np
from k_means import KMeans

class GMM:
    def __init__(self, k):
        self.k = k
        self.means = []
        self.covariances = []
        self.pis = []
        self.gammas = []

    def fit(self, data):
        """
        :params data: np.array of shape (..., dim)
                                  where dim is number of dimensions of point
        """
        self._initialize_params(data)
        while True:  # TODO: fix
            self._E_step(data)
            self._M_step(data)

    def _initialize_params(self, data):
        km = KMeans(self.k)
        km.fit(data)
        self.dim = data.shape[-1]
        _, self.means = km.predict(data)
        self.means = np.unique(self.means, axis = 0)
        self.pis = np.random.rand((k,))
        self.pis = self.pis/np.sum(self.pis)
        self.covariances = np.array([np.eye(self.dim)] * self.k)

    def _E_step(data):
        # TODO: find gammas from means, covariances, pis
        self.gammas = []

    def _M_step(data):
        # TODO: find means, covariances, pis from gammas
        pass

    def predict(self, data):
        """
        :param data: np.array of shape (..., dim)
        :return: np.array of shape (...) without dims
                         each element is integer from 0 to k-1
        """
        pass

    def get_means(self):
        return self.means.copy()

    def get_covariances(self):
        return self.covariances.copy()

    def get_pis(self):
        return self.pis.copy()


In [13]:
g = KMeans(5)

In [14]:
g.retur_means()

AttributeError: 'KMeans' object has no attribute 'retur_means'

In [15]:
a = np.load('data.npy')

In [30]:
g.fit(a)

In [31]:
c, m = g.predict(a)

In [32]:
np.unique(m, axis = 0)

array([[-7.68954908,  1.33610247],
       [-6.75782298,  4.78177533],
       [-3.1899341 ,  1.90014879],
       [ 0.63595459,  5.69909998],
       [ 4.95283136, -1.35982523]])

In [24]:
m[0]

array([ 4.95283136, -1.35982523])